In [ ]:
# Install required packages (runs automatically in Colab, fast no-op in Binder)
!pip install -q qiskit qiskit-aer qiskit-ibm-runtime pylatexenc matplotlib numpy rustworkx scipy

# Quantum Approximate Optimization Algorithm

*Nutzungsschätzung: 22 Minuta auf emm Heron r3 Prozessor (HINWEIS: Des isch nur a Schätzung. Dei Laufzeit ka variiere.)*
## Hintergrund {#Hintergrund}
Des Tutorial demonstriert d'Implementierung vo dem **Quantum Approximate Optimization Algorithm (QAOA)** – aner hybride (quanten-klassische) iterative Methode – im Kontext vo Qiskit-Patterns. Mr werde zunächst des **Maximum-Cut** (oder **Max-Cut**) Problem für a kleina Graphen löse und dann lerne, wie mer's auf Utility-Skala ausführt. Alli Hardware-Ausführunge im Tutorial sollet innerhalb vo dem Zeitlimit für de frei zugängliche Open Plan funktioniere.

Des Max-Cut-Problem isch a Optimierungsproblem, wo schwer z'löse isch (genauer gsagt isch's a *NP-hartes* Problem) und a Reihe vo verschiedene Anwendunge in Clustering, Netzwerkwissenschaft und statistischer Physik hend. Des Tutorial betrachtet a Graphen vo Knoten, wo durch Kante verbunde send, und zielt drauf ab, d'Knoten in zwei Menge z'partitioniere, so dass d'Anzahl vo de durch den Schnitt durchquerte Kante maximiert wird.

![Illustration of a max-cut problem](../docs/images/tutorials/quantum-approximate-optimization-algorithm/maxcut-illustration.avif)
## Voraussetzunge {#Voraussetzunge}

Schau vor dem Aafange vo diesem Tutorial, dass des Folgende installiert isch:
- Qiskit SDK v1.0 oder neier, mit [Visualisierungs](https://docs.quantum.ibm.com/api/qiskit/visualization)-Unterstützung
- Qiskit Runtime v0.22 oder neier (`pip install qiskit-ibm-runtime`)

Zusätzlich wird Zugang z'aner Instanz auf dr [IBM Quantum Platform](/guides/cloud-setup) benötigt. Beachtet, dass des Tutorial ned im Open Plan ausgeführt werde ka, da's Workloads mit [Sessions](https://docs.quantum.ibm.com/api/qiskit-ibm-runtime/session) ausführt, wo nur mit Premium Plan-Zugang verfügbar send.
## Einrichtung {#Einrichtung}

In [1]:
import matplotlib
import matplotlib.pyplot as plt
import rustworkx as rx
from rustworkx.visualization import mpl_draw as draw_graph
import numpy as np
from scipy.optimize import minimize
from collections import defaultdict
from typing import Sequence


from qiskit.quantum_info import SparsePauliOp
from qiskit.circuit.library import QAOAAnsatz
from qiskit.transpiler.preset_passmanagers import generate_preset_pass_manager

from qiskit_ibm_runtime import QiskitRuntimeService
from qiskit_ibm_runtime import Session, EstimatorV2 as Estimator
from qiskit_ibm_runtime import SamplerV2 as Sampler

## Teil I. QAOA im kleina Maßstab {#Teil-I-QAOA-im-kleina-Maszstab}
Dr erschte Teil vo diesem Tutorial verwendet a kleines Max-Cut-Problem, um d'Schritte z'r Lösung vo amm Optimierungsproblem mit emm Quantencomputer z'veranschauliche.

Um a bessere Kontext z'gebe, bevor des Problem auf a Quantenalgorithmus abgbildet wird, ka mr besser verstehe, wie des Max-Cut-Problem z'amm klassische kombinatorische Optimierungsproblem wird, indem mr zunächst d'Minimierung vo aner Funktion $f(x)$ betrachtet

$$
\min_{x\in {0, 1}^n}f(x),
$$

wobei d'Eingabe $x$ a Vektor isch, wo sei Komponente jedem Knoten vo amm Graphen entspreche. Dann wird jede vo diese Komponente auf entweder $0$ oder $1$ beschränkt (wo repräsentiere, ob se im Schnitt enthalte send oder ned). Dieser kleinskalige Beispielfall verwendet a Graphen mit $n=5$ Knoten.

Mr kennet a Funktion vo amm Knotenpaar $i,j$ schreibe, wo azeigt, ob d'entsprechende Kante $(i,j)$ im Schnitt liegt. Zum Beispiel isch d'Funktion $x_i + x_j - 2 x_i x_j$ genau dann 1, wenn entweder $x_i$ oder $x_j$ gleich 1 isch (was bedeutet, dass d'Kante im Schnitt liegt) und sunsch null. Des Problem, d'Kante im Schnitt z'maximiere, ka formuliert werde als

$$
\max_{x\in {0, 1}^n} \sum_{(i,j)} x_i + x_j - 2 x_i x_j,
$$

was als Minimierung umgschriebe werde ka in dr Form

$$
\min_{x\in {0, 1}^n} \sum_{(i,j)}  2 x_i x_j - x_i - x_j.
$$

Des Minimum vo $f(x)$ in diesem Fall liegt vor, wenn d'Anzahl vo de durch de Schnitt durchquerte Kante maximal isch. Wie mr sehe kennet, hend des noch nix mit Quantencomputing z'tue. Des Problem muss in ebbes umformuliert werre, wo a Quantencomputer verstehe ka.
Initialisiert des Problem, indem a Graphen mit $n=5$ Knoten erstellt wird.

In [2]:
n = 5

graph = rx.PyGraph()
graph.add_nodes_from(np.arange(0, n, 1))
edge_list = [
    (0, 1, 1.0),
    (0, 2, 1.0),
    (0, 4, 1.0),
    (1, 2, 1.0),
    (2, 3, 1.0),
    (3, 4, 1.0),
]
graph.add_edges_from(edge_list)
draw_graph(graph, node_size=600, with_labels=True)

<Image src="../docs/images/tutorials/quantum-approximate-optimization-algorithm/extracted-outputs/6ced6bea-0.avif" alt="Output of the previous code cell" />

![Output of the previous code cell](../docs/images/tutorials/quantum-approximate-optimization-algorithm/extracted-outputs/6ced6bea-0.avif)

### Schritt 1: Klassische Eingabe auf a Quantenproblem abbilden {#Schritt-1-Klassische-Eingabe-auf-a-Quantenproblem-abbilden}

Dr erschte Schritt vo dem Pattern besteht drin, des klassische Problem (Graph) auf quantenmechanische **Schaltkreise** und **Operatoren** abz'bilde. Dazu send drei Hauptschritte z'unternehme:

1. Verwendung vo aner Reihe vo mathematische Umformulierunge, um des Problem mithilfe vo dr Notation vo Quadratic Unconstrained Binary Optimization (QUBO) Probleme darzustelle.
2. Umformulierung vo dem Optimierungsproblem als Hamilton-Operator, für de dr Grundzustand dr Lösung entspricht, wo d'Kostenfunktion minimiert.
3. Erstellung vo amm Quantenschaltkreis, wo de Grundzustand vo diesem Hamilton-Operator über a Prozess ähnlich dem Quantum Annealing vorbereitet.

**Hinweis:** In dr QAOA-Methodik wollet mr letztendlich a Operator (**Hamilton-Operator**) hend, wo d'**Kostenfunktion** vo unsere hybride Algorithmus darstellt, sowie a parametrisierten Schaltkreis (**Ansatz**), wo Quantenzuständ mit Kandidatenlösunge für des Problem darstellt. Mr kennet aus diese Kandidatenzuständ samplen und se dann mit dr Kostenfunktion bewerte.

#### Graph &rarr; Optimierungsproblem {#Graph-Optimierungsproblem}

Dr erschte Schritt vo dr Abbildung isch a Notationsänderung. Des Folgende drückt des Problem in QUBO-Notation aus:

$$
\min_{x\in {0, 1}^n}x^T Q x,
$$

wobei $Q$ a $n\times n$ Matrix vo reelle Zahle isch, $n$ dr Anzahl vo de Knoten im Graphen entspricht, $x$ dr oben eiführte Vektor vo binäre Variable isch und $x^T$ d'Transponierte vo dem Vektor $x$ bezeichnet.

```
Maximize
 -2*x_0*x_1 - 2*x_0*x_2 - 2*x_0*x_4 - 2*x_1*x_2 - 2*x_2*x_3 - 2*x_3*x_4 + 3*x_0
 + 2*x_1 + 3*x_2 + 2*x_3 + 2*x_4

Subject to
  No constraints

  Binary variables (5)
    x_0 x_1 x_2 x_3 x_4
```
### Optimierungsproblem &rarr; Hamilton-Operator {#Optimierungsproblem-Hamilton-Operator}

Mr kennet dann des QUBO-Problem als **Hamilton-Operator** umformuliere (do a Matrix, wo d'Energie vo amm System darstellt):

$$
H_C=\sum_{ij}Q_{ij}Z_iZ_j + \sum_i b_iZ_i.
$$

<details>
<summary>
**Umformulierungsschritte vom QAOA-Problem zum Hamilton-Operator**
</summary>

Um z'demonstriere, wie des QAOA-Problem auf diese Weise umgschriebe werde ka, ersetzt mr zunächst d'binäre Variable $x_i$ durch a neie Satz vo Variable $z_i\in{-1, 1}$ über

$$
x_i = \frac{1-z_i}{2}.
$$

Do kennet mr sehe, dass wenn $x_i$ gleich $0$ isch, dann $z_i$ gleich $1$ sei muss. Wenn d'$x_i$ durch d'$z_i$ im Optimierungsproblem ($x^TQx$) ersetzt werde, ka a äquivalente Formulierung erhalte werde.

$$
x^TQx=\sum_{ij}Q_{ij}x_ix_j \\ =\frac{1}{4}\sum_{ij}Q_{ij}(1-z_i)(1-z_j) \\=\frac{1}{4}\sum_{ij}Q_{ij}z_iz_j-\frac{1}{4}\sum_{ij}(Q_{ij}+Q_{ji})z_i + \frac{n^2}{4}.
$$

Wenn mer jetzt $b_i=-\sum_{j}(Q_{ij}+Q_{ji})$ definiere, de Vorfaktor entferne und de konstante Term $n^2$ weglasse, erhalte mer d'beide äquivalente Formulierunge vo demselbe Optimierungsproblem.

$$
\min_{x\in{0,1}^n} x^TQx\Longleftrightarrow \min_{z\in{-1,1}^n}z^TQz + b^Tz
$$

Do hängt $b$ vo $Q$ ab. Beachtet, dass mer z'r Erlangung vo $z^TQz + b^Tz$ de Faktor 1/4 und a konstante Offset vo $n^2$ wegglasse hend, wo bei dr Optimierung kei Rolle spielet.

Um jetzt a Quantenformulierung vo dem Problem z'erhalte, erhebe mer d'Variable $z_i$ z'aner Pauli $Z$ Matrix, wie aner $2\times 2$ Matrix vo dr Form

$$
Z_i = \begin{pmatrix}1 & 0 \\ 0 & -1\end{pmatrix}.
$$

Wenn mr diese Matrize im obige Optimierungsproblem einsetzt, erhaltet mr de folgende Hamilton-Operator

$$
H_C=\sum_{ij}Q_{ij}Z_iZ_j + \sum_i b_iZ_i.
$$

*Beachtet auch, dass d'$Z$ Matrize in de Rechenraum vo dem Quantencomputer eingebettet send, des heißt in a Hilbert-Raum vo dr Größe $2^n\times 2^n$. Deswege sollet mr Terme wie $Z_iZ_j$ als des Tensorprodukt $Z_i\otimes Z_j$ verstehe, wo in de $2^n\times 2^n$ Hilbert-Raum eingebettet isch. Zum Beispiel wird in amm Problem mit fünf Entscheidungsvariable dr Term $Z_1Z_3$ als $I\otimes Z_3\otimes I\otimes Z_1\otimes I$ verstande, wobei $I$ d'$2\times 2$ Einheitsmatrix isch.*
</details>

Dieser Hamilton-Operator wird als **Kostenfunktions-Hamilton-Operator** bezeichnet. Er hend d'Eigenschaft, dass sei Grundzustand dr Lösung entspricht, wo d'**Kostenfunktion $f(x)$ minimiert**.
Um des Optimierungsproblem z'löse, muss jetzt de Grundzustand vo $H_C$ (oder a Zustand mit hoher Überlappung damit) auf dem Quantencomputer präpariert werre. Des Sampeln aus diesem Zustand wird dann mit hoher Wahrscheinlichkeit d'Lösung z'$min~f(x)$ liefera.
Betrachte mer jetzt de Hamilton-Operator $H_C$ für des **Max-Cut** Problem. Jedem Knoten vo dem Graphen wird a Qubit im Zustand $|0\rangle$ oder $|1\rangle$ zugeordnet, wobei dr Wert d'Menge angibt, z'der dr Knoten ghört. Des Ziel vo dem Problem isch's, d'Anzahl vo de Kante $(v_1, v_2)$ z'maximiere, für wo $v_1 = |0\rangle$ und $v_2 = |1\rangle$ gilt, oder umgekehrt. Wenn mer de $Z$ Operator jedem Qubit zuordnet, wobei

$$
    Z|0\rangle = |0\rangle \qquad Z|1\rangle = -|1\rangle
$$

dann ghört a Kante $(v_1, v_2)$ zum Schnitt, wenn dr Eigenwert vo $(Z_1|v_1\rangle) \cdot (Z_2|v_2\rangle) = -1$ isch; mit andere Wort, d'mit $v_1$ und $v_2$ assoziierte Qubits send unterschiedlich. Ebenso ghört $(v_1, v_2)$ ned zum Schnitt, wenn dr Eigenwert vo $(Z_1|v_1\rangle) \cdot (Z_2|v_2\rangle) = 1$ isch. Beachtet, dass uns dr genaue Qubit-Zustand, wo jedem Knoten zugeordnet isch, ned interessiert, sunsch nur, ob se über a Kante hinweg gleich send oder ned. Des Max-Cut-Problem verlangt vo uns, a Zuordnung vo de Qubits auf d'Knoten z'finde, so dass dr Eigenwert vo dem folgende Hamilton-Operator minimiert wird
$$
    H_C = \sum_{(i,j) \in e} Q_{ij} \cdot Z_i Z_j.
$$

Mit andere Wort, $b_i = 0$ für alli $i$ im Max-Cut-Problem. Dr Wert vo $Q_{ij}$ bezeichnet des Gewicht vo dr Kante. In diesem Tutorial betrachte mer a ungewichtete Graphen, des heißt $Q_{ij} = 1.0$ für alli $i, j$.

In [ ]:
def build_max_cut_paulis(
    graph: rx.PyGraph,
) -> list[tuple[str, list[int], float]]:
    """Convert the graph to Pauli list.

    This function does the inverse of `build_max_cut_graph`
    """
    pauli_list = []
    for edge in list(graph.edge_list()):
        weight = graph.get_edge_data(edge[0], edge[1])
        pauli_list.append(("ZZ", [edge[0], edge[1]], weight))
    return pauli_list


max_cut_paulis = build_max_cut_paulis(graph)
cost_hamiltonian = SparsePauliOp.from_sparse_list(max_cut_paulis, n)
print("Cost Function Hamiltonian:", cost_hamiltonian)

Cost Function Hamiltonian: SparsePauliOp(['IIIZZ', 'IIZIZ', 'ZIIIZ', 'IIZZI', 'IZZII', 'ZZIII'],
              coeffs=[1.+0.j, 1.+0.j, 1.+0.j, 1.+0.j, 1.+0.j, 1.+0.j])


#### Hamiltonian &rarr; quantum circuit

The Hamiltonian $H_C$ contains the quantum definition of your problem. Now you can create a quantum circuit that will help *sample* good solutions from the quantum computer. The QAOA is inspired by quantum annealing and applies alternating layers of operators in the quantum circuit.

The general idea is to start in the ground state of a known system, $H^{\otimes n}|0\rangle$ above, and then steer the system into the ground state of the cost operator that you are interested in. This is done by applying the operators $\exp\{-i\gamma_k H_C\}$ and $\exp\{-i\beta_k H_m\}$ with angles $\gamma_1,...,\gamma_p$ and $\beta_1,...,\beta_p~$.


The quantum circuit that you generate is **parametrized** by $\gamma_i$ and $\beta_i$, so you can try out different values of $\gamma_i$ and $\beta_i$ and sample from the resulting state.

![Circuit diagram with QAOA layers](../docs/images/tutorials/quantum-approximate-optimization-algorithm/circuit-diagram.svg)


In this case, you will try an example with one QAOA layer that contains two parameters: $\gamma_1$ and $\beta_1$.

In [4]:
circuit = QAOAAnsatz(cost_operator=cost_hamiltonian, reps=2)
circuit.measure_all()

circuit.draw("mpl")

<Image src="../docs/images/tutorials/quantum-approximate-optimization-algorithm/extracted-outputs/7bd8c6d4-f40f-4a11-a440-0b26d9021b53-0.avif" alt="Output of the previous code cell" />

In [5]:
circuit.parameters

ParameterView([ParameterVectorElement(β[0]), ParameterVectorElement(β[1]), ParameterVectorElement(γ[0]), ParameterVectorElement(γ[1])])

### Step 2: Optimize problem for quantum hardware execution

The circuit above contains a series of abstractions useful to think about quantum algorithms, but not possible to run on the hardware. To be able to run on a QPU, the circuit needs to undergo a series of operations that make up the **transpilation** or **circuit optimization** step of the pattern.

The Qiskit library offers a series of **transpilation passes** that cater to a wide range of circuit transformations. You need to make sure that your circuit is **optimized** for your purpose.

Transpilation may involves several steps, such as:

* **Initial mapping** of the qubits in the circuit (such as decision variables) to physical qubits on the device.
* **Unrolling** of the instructions in the quantum circuit to the hardware-native instructions that the backend understands.
* **Routing** of any qubits in the circuit that interact to physical qubits that are adjacent with one another.
* **Error suppression** by adding single-qubit gates to suppress noise with dynamical decoupling.


More information about transpilation is available in our [documentation](/docs/guides/transpile).

The following code transforms and optimizes the abstract circuit into a format that is ready for execution on one of devices accessible through the cloud using the **Qiskit IBM Runtime service**.

In [6]:
service = QiskitRuntimeService()
backend = service.least_busy(
    operational=True, simulator=False, min_num_qubits=127
)
print(backend)

# Create pass manager for transpilation
pm = generate_preset_pass_manager(optimization_level=3, backend=backend)

candidate_circuit = pm.run(circuit)
candidate_circuit.draw("mpl", fold=False, idle_wires=False)

<IBMBackend('test_heron_pok_1')>


<Image src="../docs/images/tutorials/quantum-approximate-optimization-algorithm/extracted-outputs/3f28a422-805c-4d3d-b5f6-62539e9133bd-1.avif" alt="Output of the previous code cell" />

### Step 3: Execute using Qiskit primitives

In the QAOA workflow, the optimal QAOA parameters are found in an iterative optimization loop, which runs a series of circuit evaluations and uses a classical optimizer to find the optimal $\beta_k$ and $\gamma_k$ parameters. This execution loop is executed via the following steps:

1. Define the initial parameters
2. Instantiate a new `Session` containing the optimization loop and the primitive used to sample the circuit
3. Once an optimal set of parameters is found, execute the circuit a final time to obtain a final distribution which will be used in the post-process step.

#### Define circuit with initial parameters
We start with arbitrary chosen parameters.

In [7]:
initial_gamma = np.pi
initial_beta = np.pi / 2
init_params = [initial_beta, initial_beta, initial_gamma, initial_gamma]

### Schritt 2: Problem für Quanten-Hardware-Ausführung optimiere {#Schritt-2-Problem-fuer-Quanten-Hardware-Ausfuehrung-optimiere}
Dr obige Schaltkreis enthält a Reihe vo Abstraktione, wo nützlich send, um über Quantenalgorithmen nachz'denke, aber auf dr Hardware ned ausführbar send. Um auf aner QPU ausgeführt werde z'könne, muss dr Schaltkreis a Reihe vo Operatione durchlaufe, wo de **Transpilations**- oder **Schaltkreis-Optimierungs**-Schritt vo dem Pattern ausmache.

D'Qiskit-Bibliothek bietet a Reihe vo **Transpilations-Pässen**, wo a breite Palette vo Schaltkreistransformatione abdeckt. Der Schaltkreis muss für de eigene Zweck **optimiert** sei.

D'Transpilation ka mehrere Schritte umfasse, wie zum Beispiel:

* **Initiales Mapping** vo de Qubits im Schaltkreis (wie Entscheidungsvariable) auf physische Qubits auf dem Gerät.
* **Unrolling** vo de Anweisunge im Quantenschaltkreis z'de hardware-native Anweisunge, wo des Backend versteht.
* **Routing** vo beliebige Qubits im Schaltkreis, wo interagiere, z'physische Qubits, wo benachbart zueinander send.
* **Fehlerunterdrückung** durch Hinzufüge vo Einzelqubit-Gates z'r Rauschunterdrückung mit dynamischer Entkopplung.

Weitere Informationen z'r Transpilation findet mr in unserer [Dokumentation](/guides/transpile).

Dr folgende Code transformiert und optimiert de abstrakte Schaltkreis in a Format, wo z'r Ausführung auf emm vo de über d'Cloud zugängliche Geräte mit dem **Qiskit IBM Runtime Service** bereit isch.

In [8]:
def cost_func_estimator(params, ansatz, hamiltonian, estimator):
    # transform the observable defined on virtual qubits to
    # an observable defined on all physical qubits
    isa_hamiltonian = hamiltonian.apply_layout(ansatz.layout)

    pub = (ansatz, isa_hamiltonian, params)
    job = estimator.run([pub])

    results = job.result()[0]
    cost = results.data.evs

    objective_func_vals.append(cost)

    return cost

In [9]:
objective_func_vals = []  # Global variable
with Session(backend=backend) as session:
    # If using qiskit-ibm-runtime<0.24.0, change `mode=` to `session=`
    estimator = Estimator(mode=session)
    estimator.options.default_shots = 1000

    # Set simple error suppression/mitigation options
    estimator.options.dynamical_decoupling.enable = True
    estimator.options.dynamical_decoupling.sequence_type = "XY4"
    estimator.options.twirling.enable_gates = True
    estimator.options.twirling.num_randomizations = "auto"

    result = minimize(
        cost_func_estimator,
        init_params,
        args=(candidate_circuit, cost_hamiltonian, estimator),
        method="COBYLA",
        tol=1e-2,
    )
    print(result)

 message: Return from COBYLA because the trust region radius reaches its lower bound.
 success: True
  status: 0
     fun: -1.6295230263157894
       x: [ 1.530e+00  1.439e+00  4.071e+00  4.434e+00]
    nfev: 26
   maxcv: 0.0


![Output of the previous code cell](../docs/images/tutorials/quantum-approximate-optimization-algorithm/extracted-outputs/3f28a422-805c-4d3d-b5f6-62539e9133bd-1.avif)

### Schritt 3: Ausführung mit Qiskit Primitives {#Schritt-3-Ausfuehrung-mit-Qiskit-Primitives}
Im QAOA-Workflow werde d'optimale QAOA-Parameter in aner iterativen Optimierungsschleife gfunde, wo a Reihe vo Schaltkreisbewertunge ausführt und a klassische Optimierer verwendet, um d'optimale $\beta_k$ und $\gamma_k$ Parameter z'finde. Diese Ausführungsschleife wird über d'folgende Schritte ausgeführt:

1. Definiere vo de initialen Parameter
2. Instanziierung vo aner neie `Session`, wo d'Optimierungsschleife und des Primitive enthält, wo zum Samplen vo dem Schaltkreis verwendet wird
3. Sobald a optimale Parametersatz gfunde isch, wird de Schaltkreis a letzts Mal ausgeführt, um a finale Verteilung z'erhalte, wo im Post-Processing-Schritt verwendet wird.
#### Schaltkreis mit initialen Parametern definiere {#Schaltkreis-mit-initialen-Parametern-definiere}
Mer beginne mit willkürlich gwählte Parametern.

In [10]:
plt.figure(figsize=(12, 6))
plt.plot(objective_func_vals)
plt.xlabel("Iteration")
plt.ylabel("Cost")
plt.show()

<Image src="../docs/images/tutorials/quantum-approximate-optimization-algorithm/extracted-outputs/e14ecc92-0.avif" alt="Output of the previous code cell" />

#### Backend und Ausführungs-Primitive definiere {#Backend-und-Ausfuehrungs-Primitive-definiere}
Verwend d'**Qiskit Runtime Primitives**, um mit IBM&reg; Backends z'interagiere. D'beide Primitives send Sampler und Estimator, und d'Wahl vo dem Primitive hängt davon ab, welche Art vo Messung auf dem Quantencomputer ausgeführt werre soll. Für d'Minimierung vo $H_C$ verwend de Estimator, da d'Messung vo dr Kostenfunktion einfach dr Erwartungswert vo $\langle H_C \rangle$ isch.
#### Ausführe {#Ausfuehre}

D'Primitives biete a Vielzahl vo [Ausführungsmodi](/guides/execution-modes) z'r Planung vo Workloads auf Quantengeräten, und a QAOA-Workflow läuft iterativ in aner Session.

![Illustration showing the behavior of Single job, Batch, and Session runtime modes.](../docs/images/tutorials/quantum-approximate-optimization-algorithm/runtime-modes.avif)

Mr kennet d'sampler-basierte Kostenfunktion in d'SciPy-Minimierungsroutine eistecke, um d'optimale Parameter z'finde.

In [11]:
optimized_circuit = candidate_circuit.assign_parameters(result.x)
optimized_circuit.draw("mpl", fold=False, idle_wires=False)

<Image src="../docs/images/tutorials/quantum-approximate-optimization-algorithm/extracted-outputs/2989e76e-4296-4dd8-b065-2b8fced064cf-0.avif" alt="Output of the previous code cell" />

In [12]:
# If using qiskit-ibm-runtime<0.24.0, change `mode=` to `backend=`
sampler = Sampler(mode=backend)
sampler.options.default_shots = 10000

# Set simple error suppression/mitigation options
sampler.options.dynamical_decoupling.enable = True
sampler.options.dynamical_decoupling.sequence_type = "XY4"
sampler.options.twirling.enable_gates = True
sampler.options.twirling.num_randomizations = "auto"

pub = (optimized_circuit,)
job = sampler.run([pub], shots=int(1e4))
counts_int = job.result()[0].data.meas.get_int_counts()
counts_bin = job.result()[0].data.meas.get_counts()
shots = sum(counts_int.values())
final_distribution_int = {key: val / shots for key, val in counts_int.items()}
final_distribution_bin = {key: val / shots for key, val in counts_bin.items()}
print(final_distribution_int)

{28: 0.0328, 11: 0.0343, 2: 0.0296, 25: 0.0308, 16: 0.0303, 27: 0.0302, 13: 0.0323, 7: 0.0312, 4: 0.0296, 9: 0.0295, 26: 0.0321, 30: 0.031, 23: 0.0324, 31: 0.0303, 21: 0.0335, 15: 0.0317, 12: 0.0309, 29: 0.0297, 3: 0.0313, 5: 0.0312, 6: 0.0274, 10: 0.0329, 22: 0.0353, 0: 0.0315, 20: 0.0326, 8: 0.0322, 14: 0.0306, 17: 0.0295, 18: 0.0279, 1: 0.0325, 24: 0.0334, 19: 0.0295}


### Step 4: Post-process and return result in desired classical format

The post-processing step interprets the sampling output to return a solution for your original problem. In this case, you are interested in the bitstring with the highest probability as this determines the optimal cut. The symmetries in the problem allow for four possible solutions, and the sampling process will return one of them with a slightly higher probability, but you can see in the plotted distribution below that four of the bitstrings are distinctively more likely than the rest.

In [13]:
# auxiliary functions to sample most likely bitstring
def to_bitstring(integer, num_bits):
    result = np.binary_repr(integer, width=num_bits)
    return [int(digit) for digit in result]


keys = list(final_distribution_int.keys())
values = list(final_distribution_int.values())
most_likely = keys[np.argmax(np.abs(values))]
most_likely_bitstring = to_bitstring(most_likely, len(graph))
most_likely_bitstring.reverse()

print("Result bitstring:", most_likely_bitstring)

Result bitstring: [0, 1, 1, 0, 1]


In [14]:
matplotlib.rcParams.update({"font.size": 10})
final_bits = final_distribution_bin
values = np.abs(list(final_bits.values()))
top_4_values = sorted(values, reverse=True)[:4]
positions = []
for value in top_4_values:
    positions.append(np.where(values == value)[0])
fig = plt.figure(figsize=(11, 6))
ax = fig.add_subplot(1, 1, 1)
plt.xticks(rotation=45)
plt.title("Result Distribution")
plt.xlabel("Bitstrings (reversed)")
plt.ylabel("Probability")
ax.bar(list(final_bits.keys()), list(final_bits.values()), color="tab:grey")
for p in positions:
    ax.get_children()[int(p[0])].set_color("tab:purple")
plt.show()

<Image src="../docs/images/tutorials/quantum-approximate-optimization-algorithm/extracted-outputs/650875e9-adbc-43bd-9505-556be2566278-0.avif" alt="Output of the previous code cell" />

![Output of the previous code cell](../docs/images/tutorials/quantum-approximate-optimization-algorithm/extracted-outputs/e14ecc92-0.avif)

Sobald d'optimale Parameter für de Schaltkreis gfunde send, kennet mr diese Parameter zuwise und d'mit de optimierte Parametern erhaltene finale Verteilung sampeln. Do sollet des *Sampler* Primitive verwendet werde, da's d'Wahrscheinlichkeitsverteilung vo Bitstring-Messunge isch, wo dem optimale Schnitt vo dem Graphen entspreche.

**Hinweis:** Des bedeutet, a Quantenzustand $\psi$ im Computer z'präpariere und en dann z'messe. A Messung wird de Zustand in a einzelne Berechnungsbasiszustand kollabiere lasse - zum Beispiel `010101110000...` - wo aner Kandidatenlösung $x$ für unser ursprüngliche Optimierungsproblem ($\max f(x)$ oder $\min f(x)$ je nach Aufgabe) entspricht.

In [15]:
# auxiliary function to plot graphs
def plot_result(G, x):
    colors = ["tab:grey" if i == 0 else "tab:purple" for i in x]
    pos, _default_axes = rx.spring_layout(G), plt.axes(frameon=True)
    rx.visualization.mpl_draw(
        G, node_color=colors, node_size=100, alpha=0.8, pos=pos
    )


plot_result(graph, most_likely_bitstring)

<Image src="../docs/images/tutorials/quantum-approximate-optimization-algorithm/extracted-outputs/33135970-8bc4-4fb2-ab87-08726a432ce4-0.avif" alt="Output of the previous code cell" />

And calculate the value of the cut:

In [16]:
def evaluate_sample(x: Sequence[int], graph: rx.PyGraph) -> float:
    assert len(x) == len(
        list(graph.nodes())
    ), "The length of x must coincide with the number of nodes in the graph."
    return sum(
        x[u] * (1 - x[v]) + x[v] * (1 - x[u])
        for u, v in list(graph.edge_list())
    )


cut_value = evaluate_sample(most_likely_bitstring, graph)
print("The value of the cut is:", cut_value)

The value of the cut is: 5


## Part II. Scale it up!

You have access to many devices with over 100 qubits on IBM Quantum&reg; Platform. Select one on which to solve Max-Cut on a 100-node weighted graph. This is a "utility-scale" problem. The steps to build the workflow are followed the same as above, but with a much larger graph.

In [17]:
n = 100  # Number of nodes in graph
graph_100 = rx.PyGraph()
graph_100.add_nodes_from(np.arange(0, n, 1))
elist = []
for edge in backend.coupling_map:
    if edge[0] < n and edge[1] < n:
        elist.append((edge[0], edge[1], 1.0))
graph_100.add_edges_from(elist)
draw_graph(graph_100, node_size=200, with_labels=True, width=1)

<Image src="../docs/images/tutorials/quantum-approximate-optimization-algorithm/extracted-outputs/590fe2ce-0.avif" alt="Output of the previous code cell" />

### Schritt 4: Nachbearbeitung und Rückgabe vo dem Ergebnis im gwünschte klassische Format {#Schritt-4-Nachbearbeitung-und-Rueckgabe-vo-dem-Ergebnis}

Dr Nachbearbeitungsschritt interpretiert d'Sampling-Ausgabe, um a Lösung für des ursprüngliche Problem zurückz'gebe. In diesem Fall send mr an dem Bitstring mit dr höchste Wahrscheinlichkeit interessiert, da dieser de optimale Schnitt bestimmt. D'Symmetrien im Problem erlaubet vier möglichi Lösunge, und dr Sampling-Prozess wird a davon mit aner etwas höhere Wahrscheinlichkeit zurückgebe, aber mr kennet in dr unten dargestellte Verteilung sehe, dass vier vo de Bitstrings deutlich wahrscheinlicher send als dr Rest.

In [18]:
max_cut_paulis_100 = build_max_cut_paulis(graph_100)

cost_hamiltonian_100 = SparsePauliOp.from_sparse_list(max_cut_paulis_100, 100)
print("Cost Function Hamiltonian:", cost_hamiltonian_100)

Cost Function Hamiltonian: SparsePauliOp(['...'],
              coeffs=[1.+0.j, 1.+0.j, 1.+0.j, 1.+0.j, 1.+0.j, 1.+0.j, 1.+0.j, 1.+0.j, 1.+0.j,
 1.+0.j, 1.+0.j, 1.+0.j, 1.+0.j, 1.+0.j, 1.+0.j, 1.+0.j, 1.+0.j, 1.+0.j,
 1.+0.j, 1.+0.j, 1.+0.j, 1.+0.j, 1.+0.j, 1.+0.j, 1.+0.j, 1.+0.j, 1.+0.j,
 1.+0.j, 1.+0.j, 1.+0.j, 1.+0.j, 1.+0.j, 1.+0.j, 1.+0.j, 1.+0.j, 1.+0.j,
 1.+0.j, 1.+0.j, 1.+0.j, 1.+0.j, 1.+0.j, 1.+0.j, 1.+0.j, 1.+0.j, 1.+0.j,
 1.+0.j, 1.+0.j, 1.+0.j, 1.+0.j, 1.+0.j, 1.+0.j, 1.+0.j, 1.+0.j, 1.+0.j,
 1.+0.j, 1.+0.j, 1.+0.j, 1.+0.j, 1.+0.j, 1.+0.j, 1.+0.j, 1.+0.j, 1.+0.j,
 1.+0.j, 1.+0.j, 1.+0.j, 1.+0.j, 1.+0.j, 1.+0.j, 1.+0.j, 1.+0.j, 1.+0.j,
 1.+0.j, 1.+0.j, 1.+0.j, 1.+0.j, 1.+0.j, 1.+0.j, 1.+0.j, 1.+0.j, 1.+0.j,
 1.+0.j, 1.+0.j, 1.+0.j, 1.+0.j, 1.+0.j, 1.+0.j, 1.+0.j, 1.+0.j, 1.+0.j,
 1.+0.j, 1.+0.j, 1.+0.j, 1.+0.j, 1.+0.j, 1.+0.j, 1.+0.j, 1.+0.j, 1.+0.j,
 1.+0.j, 1.+0.j, 1.+0.j, 1.+0.j, 1.+0.j, 1.+0.j, 1.+0.j, 1.+0.j, 1.+0.j,
 1.+0.j, 1.+0.j, 1.+0.j, 1.+0.j, 1.+0.j, 1.+0.j, 1.+0

#### Hamiltonian &rarr; quantum circuit

In [19]:
circuit_100 = QAOAAnsatz(cost_operator=cost_hamiltonian_100, reps=1)
circuit_100.measure_all()

circuit_100.draw("mpl", fold=False, scale=0.2, idle_wires=False)

<Image src="../docs/images/tutorials/quantum-approximate-optimization-algorithm/extracted-outputs/9693adfc-0.avif" alt="Output of the previous code cell" />

### Step 2: Optimize problem for quantum execution
To scale the circuit optimization step to utility-scale problems, you can take advantage of the high performance transpilation strategies introduced in Qiskit SDK v1.0. Other tools include the new transpiler service with [AI enhanced transpiler passes](/docs/guides/ai-transpiler-passes).

In [20]:
pm = generate_preset_pass_manager(optimization_level=3, backend=backend)

candidate_circuit_100 = pm.run(circuit_100)
candidate_circuit_100.draw("mpl", fold=False, scale=0.1, idle_wires=False)

<Image src="../docs/images/tutorials/quantum-approximate-optimization-algorithm/extracted-outputs/3a14e7ad-0.avif" alt="Output of the previous code cell" />

![Output of the previous code cell](../docs/images/tutorials/quantum-approximate-optimization-algorithm/extracted-outputs/650875e9-adbc-43bd-9505-556be2566278-0.avif)

#### Beste Schnitt visualisiere {#Beste-Schnitt-visualisiere}

Aus dem optimale Bitstring kennet mr dann diese Schnitt auf dem ursprüngliche Graphen visualisiere.

In [21]:
initial_gamma = np.pi
initial_beta = np.pi / 2
init_params = [initial_beta, initial_gamma]

objective_func_vals = []  # Global variable
with Session(backend=backend) as session:
    # If using qiskit-ibm-runtime<0.24.0, change `mode=` to `session=`
    estimator = Estimator(mode=session)

    estimator.options.default_shots = 1000

    # Set simple error suppression/mitigation options
    estimator.options.dynamical_decoupling.enable = True
    estimator.options.dynamical_decoupling.sequence_type = "XY4"
    estimator.options.twirling.enable_gates = True
    estimator.options.twirling.num_randomizations = "auto"

    result = minimize(
        cost_func_estimator,
        init_params,
        args=(candidate_circuit_100, cost_hamiltonian_100, estimator),
        method="COBYLA",
    )
    print(result)

 message: Return from COBYLA because the trust region radius reaches its lower bound.
 success: True
  status: 0
     fun: -3.9939191365979383
       x: [ 1.571e+00  3.142e+00]
    nfev: 29
   maxcv: 0.0


![Output of the previous code cell](../docs/images/tutorials/quantum-approximate-optimization-algorithm/extracted-outputs/33135970-8bc4-4fb2-ab87-08726a432ce4-0.avif)

Und berechnet de Wert vo dem Schnitt:

In [22]:
optimized_circuit_100 = candidate_circuit_100.assign_parameters(result.x)
optimized_circuit_100.draw("mpl", fold=False, idle_wires=False)

<Image src="../docs/images/tutorials/quantum-approximate-optimization-algorithm/extracted-outputs/1c432c2e-0.avif" alt="Output of the previous code cell" />

Finally, execute the circuit with the optimal parameters to sample from the corresponding distribution.

In [23]:
# If using qiskit-ibm-runtime<0.24.0, change `mode=` to `backend=`
sampler = Sampler(mode=backend)
sampler.options.default_shots = 10000

# Set simple error suppression/mitigation options
sampler.options.dynamical_decoupling.enable = True
sampler.options.dynamical_decoupling.sequence_type = "XY4"
sampler.options.twirling.enable_gates = True
sampler.options.twirling.num_randomizations = "auto"


pub = (optimized_circuit_100,)
job = sampler.run([pub], shots=int(1e4))

counts_int = job.result()[0].data.meas.get_int_counts()
counts_bin = job.result()[0].data.meas.get_counts()
shots = sum(counts_int.values())
final_distribution_100_int = {
    key: val / shots for key, val in counts_int.items()
}

## Teil II. Skaliert's hoch! {#Teil-II-Skaliert-hoch}

Mr hend Zugang z'vielen Geräten mit über 100 Qubits auf dr IBM Quantum&reg; Platform. Wählet oins aus, auf dem Max-Cut auf amm 100-Knoten-gewichtete Graphen glöst werde soll. Des isch a Problem im "Utility-Maßstab". D'Schritte zum Aufbau vo dem Workflow send d'gleiche wie obe, jedoch mit amm viel größere Graphen.

In [24]:
plt.figure(figsize=(12, 6))
plt.plot(objective_func_vals)
plt.xlabel("Iteration")
plt.ylabel("Cost")
plt.show()

<Image src="../docs/images/tutorials/quantum-approximate-optimization-algorithm/extracted-outputs/0fda3611-0.avif" alt="Output of the previous code cell" />

![Output of the previous code cell](../docs/images/tutorials/quantum-approximate-optimization-algorithm/extracted-outputs/590fe2ce-0.avif)

### Schritt 1: Klassische Eingabe auf a Quantenproblem abbilden {#Schritt-1-Teil-II-Klassische-Eingabe-auf-a-Quantenproblem-abbilden}
#### Graph &rarr; Hamilton-Operator {#Graph-Hamilton-Operator}
Konvertiert zunächst de Graphen, der gelöst werre soll, direkt in a Hamilton-Operator, wo für QAOA geeignet isch.

In [25]:
_PARITY = np.array(
    [-1 if bin(i).count("1") % 2 else 1 for i in range(256)],
    dtype=np.complex128,
)


def evaluate_sparse_pauli(state: int, observable: SparsePauliOp) -> complex:
    """Utility for the evaluation of the expectation value of a measured state."""
    packed_uint8 = np.packbits(observable.paulis.z, axis=1, bitorder="little")
    state_bytes = np.frombuffer(
        state.to_bytes(packed_uint8.shape[1], "little"), dtype=np.uint8
    )
    reduced = np.bitwise_xor.reduce(packed_uint8 & state_bytes, axis=1)
    return np.sum(observable.coeffs * _PARITY[reduced])


def best_solution(samples, hamiltonian):
    """Find solution with lowest cost"""
    min_cost = 1000
    min_sol = None
    for bit_str in samples.keys():
        # Qiskit use little endian hence the [::-1]
        candidate_sol = int(bit_str)
        # fval = qp.objective.evaluate(candidate_sol)
        fval = evaluate_sparse_pauli(candidate_sol, hamiltonian).real
        if fval <= min_cost:
            min_sol = candidate_sol

    return min_sol


best_sol_100 = best_solution(final_distribution_100_int, cost_hamiltonian_100)
best_sol_bitstring_100 = to_bitstring(int(best_sol_100), len(graph_100))
best_sol_bitstring_100.reverse()

print("Result bitstring:", best_sol_bitstring_100)

Result bitstring: [1, 1, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 1, 0, 1, 0, 1, 1, 0, 0, 1, 0, 1, 1, 0, 1, 0, 0, 1, 1, 1, 1, 0, 0, 1, 0, 0, 1, 0, 1, 0, 0, 0, 0, 1, 0, 1, 1, 1, 0, 0, 1, 0, 1, 0, 0, 1, 1, 1, 1, 0, 1, 1, 1, 0, 1, 0, 0, 1, 0, 1, 1, 1, 1, 0, 1, 0, 0, 1, 0, 0, 0, 0, 1, 1, 1, 0, 1, 0, 1, 0, 1, 0, 0, 1, 1]


Next, visualize the cut. Nodes of the same color belong to the same group.

In [26]:
plot_result(graph_100, best_sol_bitstring_100)

<Image src="../docs/images/tutorials/quantum-approximate-optimization-algorithm/extracted-outputs/b4a25e28-0.avif" alt="Output of the previous code cell" />

#### Hamilton-Operator &rarr; Quantenschaltkreis {#Hamilton-Operator-Quantenschaltkreis-Teil-II}

In [27]:
cut_value_100 = evaluate_sample(best_sol_bitstring_100, graph_100)
print("The value of the cut is:", cut_value_100)

The value of the cut is: 124


![Output of the previous code cell](../docs/images/tutorials/quantum-approximate-optimization-algorithm/extracted-outputs/9693adfc-0.avif)

### Schritt 2: Problem für Quantenausführung optimiere {#Schritt-2-Teil-II-Problem-fuer-Quantenausfuehrung-optimiere}
Um de Schaltkreis-Optimierungsschritt auf Utility-Scale-Probleme z'skaliere, kennet mr d'in Qiskit SDK v1.0 eiführte Hochleistungs-Transpilationsstrategien nutze. Weitere Werkzeuge umfasset de neie Transpiler-Service mit [KI-erweiterte Transpiler-Pässen](/guides/ai-transpiler-passes).

In [28]:
# auxiliary function to help plot cumulative distribution functions
def _plot_cdf(objective_values: dict, ax, color):
    x_vals = sorted(objective_values.keys(), reverse=True)
    y_vals = np.cumsum([objective_values[x] for x in x_vals])
    ax.plot(x_vals, y_vals, color=color)


def plot_cdf(dist, ax, title):
    _plot_cdf(
        dist,
        ax,
        "C1",
    )
    ax.vlines(min(list(dist.keys())), 0, 1, "C1", linestyle="--")

    ax.set_title(title)
    ax.set_xlabel("Objective function value")
    ax.set_ylabel("Cumulative distribution function")
    ax.grid(alpha=0.3)


# auxiliary function to convert bit-strings to objective values
def samples_to_objective_values(samples, hamiltonian):
    """Convert the samples to values of the objective function."""

    objective_values = defaultdict(float)
    for bit_str, prob in samples.items():
        candidate_sol = int(bit_str)
        fval = evaluate_sparse_pauli(candidate_sol, hamiltonian).real
        objective_values[fval] += prob

    return objective_values

In [29]:
result_dist = samples_to_objective_values(
    final_distribution_100_int, cost_hamiltonian_100
)

Finally, you can plot the cumulative distribution function to visualize how each sample contributes to the total probability distribution and the corresponding objective value. The horizontal spread shows the range of objective values of the samples in the final distribution. Ideally, you would see that the cumulative distribution function has "jumps" at the lower end of the objective function value axis. This would mean that few solutions with low cost have high probability of being sampled. A smooth, wide curve indicates that each sample is similarly likely, and they can have very different objective values, low or high.

In [30]:
fig, ax = plt.subplots(1, 1, figsize=(8, 6))
plot_cdf(result_dist, ax, "Eagle device")

<Image src="../docs/images/tutorials/quantum-approximate-optimization-algorithm/extracted-outputs/4381a2b3-0.avif" alt="Output of the previous code cell" />

Sobald d'optimale Parameter aus dr Ausführung vo QAOA auf dem Gerät gfunde worre send, weist mr d'Parameter dem Schaltkreis z'.